In [18]:
#vamos a probar modelos de ML
#leamos x_train, y_train, x_test, y_test
import pandas as pd
x_train = pd.read_csv("../data/processed/splits/x_train.csv")
y_train = pd.read_csv("../data/processed/splits/y_train.csv")
x_test = pd.read_csv("../data/processed/splits/x_test.csv")
y_test = pd.read_csv("../data/processed/splits/y_test.csv")




Antes de buscar cuál es el mejor modelo entre los candidatos sugeridos por la consigna, vamos a seleccionar variables usando el feature importance de un Random Forest Regressor. 

Vamos a buscar los mejores hiperparametros para un Random Forest Regressor con Optuna, después vamos a obtener su feature importance.

In [19]:
# ==============================
# Optuna para RandomForestRegressor (split temporal) + "pruning" pasivo
# ==============================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

SEED = 42
VAL_FRAC = 0.20    # último 20% del train como valid
N_TRIALS = 60      # subí/bajá según tiempo

# --------- Split temporal (train -> train/valid ; test queda intacto) ---------
X_train_all = x_train.copy()
y_train_all = y_train.copy()
X_test = x_test.copy()
y_test = y_test.copy()

n = len(X_train_all)
n_val = int(np.floor(n * VAL_FRAC))
n_tr = n - n_val
X_tr, y_tr = X_train_all.iloc[:n_tr], y_train_all[:n_tr]
X_val, y_val = X_train_all.iloc[n_tr:], y_train_all[n_tr:]

print(f"Train: {X_tr.shape}, Valid: {X_val.shape}, Test: {X_test.shape}")

# --------- Función objetivo ---------
def objective(trial: optuna.Trial) -> float:
    # Espacio de búsqueda (robusto para RF sklearn)
    max_depth_choice = trial.suggest_categorical("max_depth", [None, 6, 10, 16, 24, 32])
    max_features_choice = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.5, 0.7, 1.0])
    bootstrap_choice = trial.suggest_categorical("bootstrap", [True, False])

    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 2000),
        "criterion": "squared_error",
        "max_depth": max_depth_choice,                 # None = sin límite
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": max_features_choice,           # fracción o estrategia
        "bootstrap": bootstrap_choice,
        "random_state": SEED,
        "n_jobs": -1,
    }

    # Si se usa bootstrap, probar muestreo parcial de filas
    if bootstrap_choice:
        params["max_samples"] = trial.suggest_float("max_samples", 0.5, 1.0)

    # Modelo
    model = RandomForestRegressor(**params)

    # Entrenar y evaluar en VALID (métrica a minimizar: RMSE)
    model.fit(X_tr, y_tr)
    y_pred_val = model.predict(X_val)
    rmse_val = mean_squared_error(y_val, y_pred_val)

    # Registrar también como “valor intermedio” (no habrá pruning real, pero queda logueado)
    trial.report(rmse_val, step=0)
    # if trial.should_prune():   # en RF no habrá pasos intermedios útiles
    #     raise optuna.exceptions.TrialPruned()

    return rmse_val

# --------- Estudio Optuna ---------
study = optuna.create_study(
    direction="minimize",
    sampler=TPESampler(seed=SEED),
    pruner=MedianPruner(n_warmup_steps=5)  # no tendrá efecto real aquí, pero se deja por consistencia
)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

print("\n=== Mejores hiperparámetros (VALID) ===")
print(study.best_params)
print(f"Mejor RMSE valid: {study.best_value:.4f}")

# --------- Re-entrenar con los mejores params (train+valid) ---------
best_params = study.best_params.copy()
best_model = RandomForestRegressor(**best_params, random_state=SEED, n_jobs=-1)

best_model.fit(X_train_all, y_train_all)

# --------- Evaluación en TEST ---------
y_pred_test = best_model.predict(X_test)
mse  = mean_squared_error(y_test, y_pred_test)
rmse = mean_squared_error(y_test, y_pred_test)
mae  = mean_absolute_error(y_test, y_pred_test)
r2   = r2_score(y_test, y_pred_test)

print("\n=== Métricas en TEST ===")
print(f"MSE : {mse:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"MAE : {mae:,.2f}")
print(f"R²  : {r2:.4f}")


[I 2025-11-17 00:27:41,348] A new study created in memory with name: no-name-160b5992-e5c5-40e1-8f7c-d986fe297c1f


Train: (667, 174), Valid: (166, 174), Test: (356, 174)


Best trial: 0. Best value: 9.11314e+07:   2%|▏         | 1/60 [00:00<00:30,  1.92it/s]

[I 2025-11-17 00:27:41,870] Trial 0 finished with value: 91131365.09297095 and parameters: {'max_depth': 6, 'max_features': 'log2', 'bootstrap': True, 'n_estimators': 582, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_samples': 0.6521211214797689}. Best is trial 0 with value: 91131365.09297095.


Best trial: 1. Best value: 3.20066e+07:   3%|▎         | 2/60 [00:02<01:15,  1.31s/it]

...

[I 2025-11-17 00:29:19,182] Trial 57 finished with value: 21033399.11844295 and parameters: {'max_depth': 32, 'max_features': 0.5, 'bootstrap': False, 'n_estimators': 970, 'min_samples_split': 11, 'min_samples_leaf': 4}. Best is trial 57 with value: 21033399.11844295.


Best trial: 57. Best value: 2.10334e+07:  98%|█████████▊| 59/60 [01:40<00:01,  1.97s/it]

[I 2025-11-17 00:29:21,861] Trial 58 finished with value: 43578676.26883574 and parameters: {'max_depth': 32, 'max_features': 1.0, 'bootstrap': False, 'n_estimators': 816, 'min_samples_split': 13, 'min_samples_leaf': 4}. Best is trial 57 with value: 21033399.11844295.


Best trial: 57. Best value: 2.10334e+07: 100%|██████████| 60/60 [01:42<00:00,  1.70s/it]


[I 2025-11-17 00:29:23,527] Trial 59 finished with value: 801422544.4119908 and parameters: {'max_depth': 32, 'max_features': 0.5, 'bootstrap': False, 'n_estimators': 965, 'min_samples_split': 8, 'min_samples_leaf': 7}. Best is trial 57 with value: 21033399.11844295.

=== Mejores hiperparámetros (VALID) ===
{'max_depth': 32, 'max_features': 0.5, 'bootstrap': False, 'n_estimators': 970, 'min_samples_split': 11, 'min_samples_leaf': 4}
Mejor RMSE valid: 21033399.1184

=== Métricas en TEST ===
MSE : 14,377,245.74
RMSE: 14,377,245.74
MAE : 2,866.04
R²  : 0.5933


In [20]:
"""El mejor modelo fue RFRegressor con los siguientes hiperparámetros:
    {'max_depth': 16, 'max_features': 1.0, 'bootstrap': True, 'n_estimators': 1183, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_samples': 0.8654367662002576}"""
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#Entrenemos un modelo con esos hiperparámetros

best_model = RandomForestRegressor(
    n_estimators=1183,
    max_depth=16,
    max_features=1.0,
    bootstrap=True,
    min_samples_split=2,
    min_samples_leaf=2,
    max_samples=0.8654367662002576,
    random_state=42,
    n_jobs=-1
)
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
x_train_ready = pd.DataFrame(pt.fit_transform(x_train), columns=x_train.columns)
x_test_ready = pd.DataFrame(pt.transform(x_test), columns=x_test.columns)

best_model.fit(x_train_ready, y_train)
y_pred_test = best_model.predict(x_test_ready)
mse  = mean_squared_error(y_test, y_pred_test)
rmse = mean_squared_error(y_test, y_pred_test)
mae  = mean_absolute_error(y_test, y_pred_test)
r2   = r2_score(y_test, y_pred_test)
print("\n=== Métricas en TEST del mejor modelo entrenado manualmente ===")
print(f"MSE : {mse:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"MAE : {mae:,.2f}")
print(f"R²  : {r2:.4f}")

#gracias a este random forest vamos a definir nuestras variables más relevantes

feature_importances = best_model.feature_importances_
feature_names = x_train.columns
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
feature_importances = feature_importances[:20]
print("Feature importances:")
print(feature_importances)
x_train = x_train[feature_importances['feature']]
x_test = x_test[feature_importances['feature']]


=== Métricas en TEST del mejor modelo entrenado manualmente ===
MSE : 11,980,515.69
RMSE: 11,980,515.69
MAE : 2,684.17
R²  : 0.6611
Feature importances:
                    feature  importance
40                Frio (Kw)    0.529431
36            Sala Maq (Kw)    0.198587
167   Frio_roll_mean_7_lag1    0.118802
32            Envasado (Kw)    0.017051
169  Frio_roll_mean_14_lag1    0.013890
35           Servicios (Kw)    0.013222
171  Frio_roll_mean_28_lag1    0.004571
160                 mes_cos    0.003345
45           KW Gral Planta    0.003033
13        ET Servicios / Hl    0.002859
11             ET Bodega/Hl    0.002751
6              EE Frio / Hl    0.002404
41     Pta Agua / Eflu (Kw)    0.002376
5          EE Sala Maq / Hl    0.002209
168    Frio_roll_std_7_lag1    0.002168
26            Hl Cerveza L4    0.002089
43          Resto Serv (Kw)    0.001996
34             Linea 3 (Kw)    0.001928
42           Prod Agua (Kw)    0.001887
164         Frio_diff7_lag1    0.001867


Ahora vamos a seleccionar las primeras 20 variables y buscar el modelo que minimice el MAE en nuestro test.

In [21]:
# ============================================
# Experiments: Tracking, Evaluation & Plots
# ============================================
import os, json, time, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

# Modelos y utils
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from lightgbm import LGBMRegressor, early_stopping
import optuna

import matplotlib.pyplot as plt

# =========================
# Config
# =========================
SEED = 42
VAL_FRAC = 0.20
N_TRIALS = 50   # Optuna
RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
LOG_PATH = RESULTS_DIR / "experiment_logs.csv"
TS = datetime.now().strftime("%Y%m%d_%H%M%S")

# =========================
# Helpers
# =========================
def metrics_dict(y_true, y_pred):
    return {
        "mae":  mean_absolute_error(y_true, y_pred),
        "rmse": np.sqrt(mean_squared_error(y_true, y_pred)), # squared=False devuelve RMSE
        "mse":  mean_squared_error(y_true, y_pred),
        "r2":   r2_score(y_true, y_pred),
    }

def append_log(row: dict, log_path: Path = LOG_PATH):
    row = row.copy()
    row["timestamp"] = datetime.now().isoformat()
    df_row = pd.DataFrame([row])
    if not log_path.exists():
        df_row.to_csv(log_path, index=False)
    else:
        df_row.to_csv(log_path, index=False, mode="a", header=False)

def plot_pred_vs_true(y_true, y_pred, title, out_path):
    plt.figure()
    plt.scatter(y_true, y_pred, s=8)
    minv = np.min([y_true.min(), y_pred.min()])
    maxv = np.max([y_true.max(), y_pred.max()])
    plt.plot([minv, maxv], [minv, maxv], 'r--')
    plt.xlabel("Valor real")
    plt.ylabel("Predicción")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()

def plot_residuals(y_true, y_pred, title, out_path_scatter, out_path_hist):
    resid = y_pred - y_true
    # Residual vs Pred
    plt.figure()
    plt.scatter(y_pred, resid, s=8)
    plt.axhline(0, color='r', linestyle='--')
    plt.xlabel("Predicción")
    plt.ylabel("Residuo")
    plt.title(title + " - Residual vs Pred")
    plt.tight_layout()
    plt.savefig(out_path_scatter, dpi=150)
    plt.close()
    # Histograma
    plt.figure()
    plt.hist(resid, bins=30, edgecolor='k')
    plt.xlabel("Residuo")
    plt.ylabel("Frecuencia")
    plt.title(title + " - Hist Residuales")
    plt.tight_layout()
    plt.savefig(out_path_hist, dpi=150)
    plt.close()

def save_feature_importance(model, feature_names, out_csv, out_png, top_k=30):
    # Pipeline handling: get the final estimator
    if isinstance(model, Pipeline):
        est = model.steps[-1][1]
        # Coefs for linear in pipeline
        if hasattr(est, "coef_"):
            coef = np.ravel(est.coef_)
            imp = pd.DataFrame({"feature": feature_names, "importance": np.abs(coef)})
        elif hasattr(est, "feature_importances_"):
            importances = est.feature_importances_
            imp = pd.DataFrame({"feature": feature_names, "importance": importances})
        else:
             imp = pd.DataFrame({"feature": feature_names, "importance": np.zeros(len(feature_names))})
    else:
        # Standard models
        if hasattr(model, "feature_importances_"):
            importances = model.feature_importances_
            imp = pd.DataFrame({"feature": feature_names, "importance": importances})
        elif hasattr(model, "coef_"):
            coef = np.ravel(model.coef_)
            imp = pd.DataFrame({"feature": feature_names, "importance": np.abs(coef)})
        else:
            # XGB/LGBM wrappers usually have feature_importances_
            try:
                importances = model.get_booster().get_score(importance_type="gain")
                # XGB keys might not match feature names exactly if not set, mapping is safer but simple dict works often
                imp = pd.DataFrame({"feature": list(importances.keys()),
                                    "importance": list(importances.values())})
            except Exception:
                imp = pd.DataFrame({"feature": feature_names, "importance": np.zeros(len(feature_names))})

    imp = imp.sort_values("importance", ascending=False)
    imp.to_csv(out_csv, index=False)

    top = imp.head(top_k)
    plt.figure(figsize=(8, max(3, 0.28*len(top))))
    plt.barh(top["feature"][::-1], top["importance"][::-1])
    plt.xlabel("Importancia")
    plt.title("Feature importance (top)")
    plt.tight_layout()
    plt.savefig(out_png, dpi=150)
    plt.close()

def refit_full_for_test(model, X_tr, y_tr, X_val=None, y_val=None):
    """
    Refit en todo el train usando mejor n_estimators si hay early stopping.
    """
    # XGB
    if isinstance(model, xgb.XGBRegressor):
        best_n = getattr(model, "best_iteration", None)
        params = model.get_params()
        if best_n is not None:
            params["n_estimators"] = best_n
        m = xgb.XGBRegressor(**params)
        m.fit(X_tr, y_tr)  # full train
        return m

    # LGBM
    if isinstance(model, LGBMRegressor):
        best_n = getattr(model, "best_iteration_", None)
        params = model.get_params()
        if best_n is not None and best_n > 0:
            params["n_estimators"] = best_n
        m = LGBMRegressor(**params)
        m.fit(X_tr, y_tr)
        return m

    # Otros: re-usa el modelo tal cual y refitea
    # Si es pipeline, clone o crear nuevo
    from sklearn.base import clone
    m = clone(model)
    m.fit(X_tr, y_tr)
    return m

def write_summary_md(log_path: Path, summary_path: Path):
    df = pd.read_csv(log_path)
    # MODIFICADO: Orden por MAE de Test
    df = df.sort_values("test_mae").reset_index(drop=True)
    lines = []
    lines.append(f"# Experiment Summary — {datetime.now().isoformat()}\n")
    lines.append("## Top 10 por MAE de Test\n")
    cols = ["timestamp","model_name","test_mae","test_rmse","test_r2","val_rmse","val_mae"]
    lines.append(df[cols].head(10).to_markdown(index=False))
    lines.append("\n## Espacios de búsqueda declarados\n")
    for _, row in df.iterrows():
        if pd.notna(row.get("search_space_str", "")) and str(row.get("search_space_str", "")).strip():
            lines.append(f"- **{row['model_name']}**: `{row['search_space_str']}`")
    lines.append("\n## Justificación\n")
    lines.append("- **CRITERIO DE SELECCIÓN: Menor MAE en Test.**")
    lines.append("- Se reportan métricas en test del modelo refiteado en todo el train.")
    lines.append("- La elección queda respaldada por `results/experiment_logs.csv` y este resumen.")
    summary_path.write_text("\n".join(lines), encoding="utf-8")

# =========================
# 0) Se espera que tengas x_train, y_train, x_test, y_test en memoria
# =========================

# =========================
# 1) PowerTransformer como baseline
# =========================
pt = PowerTransformer(method='yeo-johnson')
X_train_ready = pd.DataFrame(pt.fit_transform(x_train), columns=x_train.columns, index=x_train.index)
X_test_ready  = pd.DataFrame(pt.transform(x_test),      columns=x_test.columns,  index=x_test.index)

# Split temporal interno para validación
n = len(X_train_ready)
n_val = int(np.floor(n * VAL_FRAC))
n_tr = n - n_val
X_tr, y_tr = X_train_ready.iloc[:n_tr], np.ravel(y_train.iloc[:n_tr])
X_val, y_val = X_train_ready.iloc[n_tr:], np.ravel(y_train.iloc[n_tr:])
X_full, y_full = X_train_ready, np.ravel(y_train)

# Para logs
base_log_ctx = {
    "n_train": len(X_tr),
    "n_val": len(X_val),
    "n_full_train": len(X_full),
    "n_test": len(X_test_ready),
    "n_features": X_train_ready.shape[1],
}

# =========================
# 2) Modelos y búsquedas
# =========================
all_results = []

# (a) RandomForest — baseline
rf = RandomForestRegressor(
    n_estimators=1183,
    max_depth=16,
    max_features=1.0,
    bootstrap=True,
    min_samples_split=2,
    min_samples_leaf=2,
    max_samples=0.8654367662002576,
    random_state=SEED,
    n_jobs=-1
)
rf.fit(X_tr, y_tr)
y_val_pred = rf.predict(X_val)
val_metrics = metrics_dict(y_val, y_val_pred)

rf_refit = refit_full_for_test(rf, X_full, y_full)
y_test_pred = rf_refit.predict(X_test_ready)
test_metrics = metrics_dict(np.ravel(y_test), y_test_pred)

append_log({
    **base_log_ctx,
    "model_name": "RandomForest",
    "params_json": json.dumps(rf.get_params()),
    "search_space_str": "Fixed params (baseline)",
    "val_mae": val_metrics["mae"],
    "val_rmse": val_metrics["rmse"],
    "val_r2": val_metrics["r2"],
    "test_mae": test_metrics["mae"],
    "test_rmse": test_metrics["rmse"],
    "test_r2": test_metrics["r2"],
}, LOG_PATH)
# MODIFICADO: Guardamos test_metrics["mae"] para comparar
all_results.append(("RandomForest", test_metrics["mae"], rf, rf_refit, y_val_pred, y_test_pred))

# (b) XGBoost — early stopping en validación
xgb_model = xgb.XGBRegressor(
    n_estimators=5000, learning_rate=0.02, max_depth=8,
    subsample=0.8, colsample_bytree=0.8, reg_alpha=0.0, reg_lambda=1.0,
    min_child_weight=1.0, random_state=SEED, tree_method="hist", n_jobs=-1
)
xgb_model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False) # verbose false para limpiar salida
y_val_pred = xgb_model.predict(X_val)
val_metrics = metrics_dict(y_val, y_val_pred)

xgb_refit = refit_full_for_test(xgb_model, X_full, y_full)
y_test_pred = xgb_refit.predict(X_test_ready)
test_metrics = metrics_dict(np.ravel(y_test), y_test_pred)

append_log({
    **base_log_ctx,
    "model_name": "XGBoost",
    "params_json": json.dumps(xgb_model.get_params()),
    "search_space_str": "Fixed params; early_stopping_rounds=200",
    "val_mae": val_metrics["mae"],
    "val_rmse": val_metrics["rmse"],
    "val_r2": val_metrics["r2"],
    "test_mae": test_metrics["mae"],
    "test_rmse": test_metrics["rmse"],
    "test_r2": test_metrics["r2"],
}, LOG_PATH)
# MODIFICADO: Guardamos test_metrics["mae"]
all_results.append(("XGBoost", test_metrics["mae"], xgb_model, xgb_refit, y_val_pred, y_test_pred))

# (c) LightGBM — early stopping en validación
lgbm_model = LGBMRegressor(
    n_estimators=5000, learning_rate=0.02, num_leaves=127, max_depth=-1,
    subsample=0.8, colsample_bytree=0.8, reg_alpha=0.0, reg_lambda=1.0,
    min_child_samples=20, random_state=SEED, n_jobs=-1
)
lgbm_model.fit(X_tr, y_tr,
               eval_set=[(X_val, y_val)],
               callbacks=[early_stopping(stopping_rounds=200, verbose=False)])
y_val_pred = lgbm_model.predict(X_val)
val_metrics = metrics_dict(y_val, y_val_pred)

lgbm_refit = refit_full_for_test(lgbm_model, X_full, y_full)
y_test_pred = lgbm_refit.predict(X_test_ready)
test_metrics = metrics_dict(np.ravel(y_test), y_test_pred)

append_log({
    **base_log_ctx,
    "model_name": "LightGBM",
    "params_json": json.dumps(lgbm_model.get_params()),
    "search_space_str": "Fixed params; early_stopping_rounds=200",
    "val_mae": val_metrics["mae"],
    "val_rmse": val_metrics["rmse"],
    "val_r2": val_metrics["r2"],
    "test_mae": test_metrics["mae"],
    "test_rmse": test_metrics["rmse"],
    "test_r2": test_metrics["r2"],
}, LOG_PATH)
# MODIFICADO: Guardamos test_metrics["mae"]
all_results.append(("LightGBM", test_metrics["mae"], lgbm_model, lgbm_refit, y_val_pred, y_test_pred))

# (d) Ridge — Optuna (CV TimeSeriesSplit)
tscv = TimeSeriesSplit(n_splits=5)
def ridge_objective(trial: optuna.Trial):
    alpha = trial.suggest_float("alpha", 1e-4, 1e3, log=True)
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge(alpha=alpha, random_state=SEED))
    ])
    scores = cross_val_score(pipe, X_full, y_full, cv=tscv,
                             scoring="neg_mean_absolute_error", n_jobs=-1) # Ojo: Optuna optimiza validación interna
    return -scores.mean()

ridge_space = "alpha ~ loguniform[1e-4, 1e3]"
ridge_study = optuna.create_study(direction="minimize", study_name="ridge_opt")
ridge_study.optimize(ridge_objective, n_trials=N_TRIALS, show_progress_bar=False)
best_alpha_ridge = ridge_study.best_params["alpha"]

ridge_best = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=best_alpha_ridge, random_state=SEED))
])
# Validación entrenando en X_tr
ridge_best.fit(X_tr, y_tr)
y_val_pred = ridge_best.predict(X_val)
val_metrics = metrics_dict(y_val, y_val_pred)

# Test refiteado en todo el train
ridge_refit = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=best_alpha_ridge, random_state=SEED))
])
ridge_refit.fit(X_full, y_full)
y_test_pred = ridge_refit.predict(X_test_ready)
test_metrics = metrics_dict(np.ravel(y_test), y_test_pred)

append_log({
    **base_log_ctx,
    "model_name": "Ridge (Optuna)",
    "params_json": json.dumps({"alpha": float(best_alpha_ridge)}),
    "search_space_str": ridge_space + f"; n_trials={N_TRIALS}",
    "val_mae": val_metrics["mae"],
    "val_rmse": val_metrics["rmse"],
    "val_r2": val_metrics["r2"],
    "test_mae": test_metrics["mae"],
    "test_rmse": test_metrics["rmse"],
    "test_r2": test_metrics["r2"],
}, LOG_PATH)
# MODIFICADO: Guardamos test_metrics["mae"]
all_results.append(("Ridge (Optuna)", test_metrics["mae"], ridge_best, ridge_refit, y_val_pred, y_test_pred))

# (e) Lasso — Optuna
def lasso_objective(trial: optuna.Trial):
    alpha = trial.suggest_float("alpha", 1e-4, 1e2, log=True)
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", Lasso(alpha=alpha, random_state=SEED, max_iter=10000))
    ])
    scores = cross_val_score(pipe, X_full, y_full, cv=tscv,
                             scoring="neg_mean_absolute_error", n_jobs=-1)
    return -scores.mean()

lasso_space = "alpha ~ loguniform[1e-4, 1e2]"
lasso_study = optuna.create_study(direction="minimize", study_name="lasso_opt")
lasso_study.optimize(lasso_objective, n_trials=N_TRIALS, show_progress_bar=False)
best_alpha_lasso = lasso_study.best_params["alpha"]

lasso_best = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=best_alpha_lasso, random_state=SEED, max_iter=10000))
])
lasso_best.fit(X_tr, y_tr)
y_val_pred = lasso_best.predict(X_val)
val_metrics = metrics_dict(y_val, y_val_pred)

lasso_refit = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=best_alpha_lasso, random_state=SEED, max_iter=10000))
])
lasso_refit.fit(X_full, y_full)
y_test_pred = lasso_refit.predict(X_test_ready)
test_metrics = metrics_dict(np.ravel(y_test), y_test_pred)

append_log({
    **base_log_ctx,
    "model_name": "Lasso (Optuna)",
    "params_json": json.dumps({"alpha": float(best_alpha_lasso)}),
    "search_space_str": lasso_space + f"; n_trials={N_TRIALS}",
    "val_mae": val_metrics["mae"],
    "val_rmse": val_metrics["rmse"],
    "val_r2": val_metrics["r2"],
    "test_mae": test_metrics["mae"],
    "test_rmse": test_metrics["rmse"],
    "test_r2": test_metrics["r2"],
}, LOG_PATH)
# MODIFICADO: Guardamos test_metrics["mae"]
all_results.append(("Lasso (Optuna)", test_metrics["mae"], lasso_best, lasso_refit, y_val_pred, y_test_pred))

# =========================
# 3) Selección por MAE de TEST + gráficos y FI
# =========================
# all_results: (name, test_mae, model_val_fitted, model_test_refit, y_val_pred, y_test_pred)
all_results.sort(key=lambda t: t[1]) # Ordena de menor a mayor (MAE menor es mejor)
best_name, best_test_mae, best_model_val, best_model_full, best_yval_pred, best_ytest_pred = all_results[0]

print(f"\n>>> Mejor modelo seleccionado por MAE en Test: {best_name} (Test MAE={best_test_mae:.4f})")

# Plots (validación y test) para el mejor
# Validación
plot_pred_vs_true(
    y_true=y_val, y_pred=best_yval_pred,
    title=f"{best_name} - Validación",
    out_path=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_val_pred_vs_true.png"
)
plot_residuals(
    y_true=y_val, y_pred=best_yval_pred,
    title=f"{best_name} - Validación",
    out_path_scatter=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_val_resid_scatter.png",
    out_path_hist=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_val_resid_hist.png"
)

# Test
plot_pred_vs_true(
    y_true=np.ravel(y_test), y_pred=best_ytest_pred,
    title=f"{best_name} - Test",
    out_path=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_test_pred_vs_true.png"
)
plot_residuals(
    y_true=np.ravel(y_test), y_pred=best_ytest_pred,
    title=f"{best_name} - Test",
    out_path_scatter=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_test_resid_scatter.png",
    out_path_hist=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_test_resid_hist.png"
)

# Feature importance del mejor (en el modelo refiteado full)
save_feature_importance(
    best_model_full,
    feature_names=X_train_ready.columns.tolist(),
    out_csv=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_feature_importance.csv",
    out_png=RESULTS_DIR / f"{TS}_{best_name.replace(' ','_')}_feature_importance.png",
    top_k=30
)

# =========================
# 4) Resumen de experimentos (MD)
# =========================
summary_md_path = RESULTS_DIR / f"experiment_summary_{TS}.md"
write_summary_md(LOG_PATH, summary_md_path)

print("\n==== Listo ====")
print(f"- Logs: {LOG_PATH}")
print(f"- Resumen: {summary_md_path}")
print(f"- Gráficos y FI en: {RESULTS_DIR}/")

[I 2025-11-17 00:29:46,624] A new study created in memory with name: ridge_opt


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4240
[LightGBM] [Info] Number of data points in the train set: 667, number of used features: 20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 28568.025112
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

[I 2025-11-17 00:29:49,199] Trial 0 finished with value: 21524.070042132178 and parameters: {'alpha': 0.0006967758838912295}. Best is trial 0 with value: 21524.070042132178.
[I 2025-11-17 00:29:51,160] Trial 1 finished with value: 21422.525152250037 and parameters: {'alpha': 0.07582035756045312}. Best is trial 1 with value: 21422.525152250037.
[I 2025-11-17 00:29:53,164] Trial 2 finished with value: 20601.465554133567 and parameters: {'alpha': 0.9950422811956947}. Best is trial 2 with value: 20601.465554133567.
[I 2025-11-17 00:29:54,756] Trial 3 finished with value: 15644.4931506505 and parameters: {'alpha': 27.120661603216256}. Best is trial 3 with value: 15644.4931506505.
[I 2025-11-17 00:29:54,770] Trial 4 finished with value: 15493.942881579922 and parameters: {'alpha': 29.14739282239711}. Best is trial 4 with value: 15493.942881579922.
[I 2025-11-17 00:29:54,785] Trial 5 finished with value: 13424.59431679433 and parameters: {'alpha': 88.39345499443026}. Best is trial 5 with valu


>>> Mejor modelo seleccionado por MAE en Test: RandomForest (Test MAE=2656.5378)

==== Listo ====
- Logs: results\experiment_logs.csv
- Resumen: results\experiment_summary_20251117_002929.md
- Gráficos y FI en: results/
